In [ ]:
import open3d as o3d
import numpy as np
import os
from pytorch3d.loss import chamfer_distance
import torch

boat = o3d.io.read_point_cloud("pcdObjects/boat.pcd")
car = o3d.io.read_point_cloud("pcdObjects/car.pcd")
table = o3d.io.read_point_cloud("pcdObjects/table.pcd")
laptop = o3d.io.read_point_cloud("pcdObjects/laptop.pcd")
plane = o3d.io.read_point_cloud("pcdObjects/plane.pcd")
trashcan = o3d.io.read_point_cloud("pcdObjects/trashcan.pcd")

pcds = []
pcds.append(table)

R = laptop.get_rotation_matrix_from_xyz((np.pi/2, np.pi/2, 0))
laptop.rotate(R)
laptop.scale(20, center=laptop.get_center())
laptop.translate((85, 90, 32))
pcds.append(laptop)

R = car.get_rotation_matrix_from_xyz((np.pi/2, -np.pi/4, 0))
car.rotate(R)
car.translate((95, 90, 30))
car.scale(10, center=car.get_center())
pcds.append(car)

R = trashcan.get_rotation_matrix_from_xyz((np.pi/2, 0, 0))
trashcan.rotate(R)
trashcan.translate((98, 106, 33.3))
trashcan.scale(13, center=trashcan.get_center())
pcds.append(trashcan)

R = plane.get_rotation_matrix_from_xyz((np.pi/2, np.pi/4, 0))
plane.rotate(R)
plane.translate((85, 105, 29.5))
plane.scale(30, center=plane.get_center())
pcds.append(plane)

R = boat.get_rotation_matrix_from_xyz((np.pi/2, -np.pi/2, 0))
boat.rotate(R)
boat.translate((88, 75, 30))
boat.scale(20, center=boat.get_center())
pcds.append(boat)

singlePcd = o3d.geometry.PointCloud()
for i in pcds:
    singlePcd += i



In [ ]:
newPcds = []
diameter = np.linalg.norm(np.asarray(singlePcd.get_max_bound()) - np.asarray(singlePcd.get_min_bound()))
radius = diameter * 100

camera = [0, 0, diameter]
_, pt_map = singlePcd.hidden_point_removal(camera, radius)
pcd1 = singlePcd.select_by_index(pt_map)
# o3d.visualization.draw_geometries([pcd1])
newPcds.append(pcd1)

In [ ]:
camera = [0, diameter, 0]
_, pt_map = singlePcd.hidden_point_removal(camera, radius)
pcd2 = singlePcd.select_by_index(pt_map)
# o3d.visualization.draw_geometries([pcd2])
newPcds.append(pcd2)

In [ ]:
camera = [diameter, 0, 3*diameter]
_, pt_map = singlePcd.hidden_point_removal(camera, radius)
pcd3 = singlePcd.select_by_index(pt_map)
# o3d.visualization.draw_geometries([pcd3])
newPcds.append(pcd3)

In [ ]:
camera = [diameter, 0, diameter]
_, pt_map = singlePcd.hidden_point_removal(camera, radius)
pcd4 = singlePcd.select_by_index(pt_map)
# o3d.visualization.draw_geometries([pcd4])
newPcds.append(pcd4)

In [ ]:
camera = [diameter, 2*diameter, diameter]
_, pt_map = singlePcd.hidden_point_removal(camera, radius)
pcd5 = singlePcd.select_by_index(pt_map)
# o3d.visualization.draw_geometries([pcd5])
newPcds.append(pcd5)

In [ ]:
camera = [diameter, diameter, 0]
_, pt_map = singlePcd.hidden_point_removal(camera, radius)
pcd6 = singlePcd.select_by_index(pt_map)
# o3d.visualization.draw_geometries([pcd6])
newPcds.append(pcd6)

In [ ]:
reconstructedPcd = o3d.geometry.PointCloud()
for i in newPcds:
    reconstructedPcd += i

In [ ]:
aray = np.asarray(reconstructedPcd.points)
array = np.asarray(aray)
x = torch.FloatTensor(aray)
x.unsqueeze_(0)
print(x.shape)
aray2 = np.asarray(singlePcd.points)
y = torch.FloatTensor(aray2)
y.unsqueeze_(0)
print(y.shape)
chamfer_distance(y, x)

- Chamfers Distance tells us how much the point clouds differ from each other. It's basically a loss function for point clouds. If CD == 0, then it means that the point clouds are identical.
- The number of points in both the point clouds "reconstructedPcd" and "singlePcd" are different as the reconstructed point cloud is just a merged version of point clouds from different views. It does not completely reconstruct the scene. The scene has holes and is not completed.
- With increase in viewpoints, the scene gets completed and the size of holes/ number of holes in the reconstructed pcd decrease. Hence, the CD decreases with increases in viewpoints.
- Yes, we can choose the viewpoints such that the CD decreases. If we are limited to a finite number of viewpoints, then choosing viewpoints such that maximum number of unique points get covered will result in a lower CD.
